In [59]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime
from tqdm.notebook import tqdm
tqdm.pandas()
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [4]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [5]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [6]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [7]:
ME_QUERY = \
    f"""
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME,
    P.UPDATE_TMSTP,
    P.ACTIVE_IND
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [8]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 9
    """
entity_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [9]:
NAME_QUERY = \
    """
    SELECT
    FIRST_NM, MIDDLE_NM, LAST_NM, SUFFIX, LABEL_NM, NM_TYPE, PARTY_ID
    FROM
    AMAEDW.PERSON_NM
    WHERE
    THRU_DT IS NULL
    AND
    NM_TYPE='LN'
    """
NAMES = pd.read_sql(con=AMAEDW, sql=NAME_QUERY)

In [10]:
PERSON_QUERY = \
    """
    SELECT
    BIRTH_DT, BIRTH_CITY_NM, BIRTH_STATE_ID, BIRTH_COUNTRY_ID, GENDER_CD, MORTALITY_STS_CD, DEATH_DT, MED_EDU_NBR, PARTY_ID
    FROM
    AMAEDW.PERSON
    WHERE
    ACTIVE_IND = 'Y'
    """
PERSONS = pd.read_sql(con=AMAEDW, sql=PERSON_QUERY)

In [11]:
IDS = pd.merge(entity_ids, party_ids, on='PARTY_ID')
test = pd.merge(IDS, ov_me, left_on='ME', right_on='medical_education_number', how='right')
test_cont = pd.merge(test, NAMES, on='PARTY_ID',how='left').drop_duplicates()
ALL = pd.merge(test_cont, PERSONS, on='PARTY_ID',how='left').drop_duplicates()

In [14]:
ALL

1419935

In [12]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
dead_mes = list(ALL[(ALL.MORTALITY_STS_CD=='C')].ME)
all_mes = list(ALL.ME)
us_mes = list(ALL[(ALL.BIRTH_COUNTRY_ID==6705)].ME)
elements = [
    {'data_element': 'Party ID',
     'element':'PARTY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Entity ID',
     'element':'ENTITY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'First Name',
     'element':'FIRST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Middle Name',
     'element':'MIDDLE_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Last Name',
     'element':'LAST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Suffix',
     'element':'SUFFIX',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Label Name',
     'element':'LABEL_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Type',
     'element':'NM_TYPE',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Person Type',
     'element':'type',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Race/Ethnicity',
     'element':'race_ethnicity',
    'null': 'Unknown',
    'universe': all_mes},
    {'data_element': 'Birth Date',
     'element':'BIRTH_DT',
    'null': datetime.date(1900, 1, 1),
    'universe':all_mes},
    {'data_element': 'Birth City',
     'element':'BIRTH_CITY_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Birth State',
     'element':'BIRTH_STATE_ID',
    'null': '',
    'universe': us_mes},
    {'data_element': 'Birth Country',
     'element':'BIRTH_COUNTRY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Gender',
     'element':'GENDER_CD',
    'null': '',
    'universe':all_mes},
    {'data_element': 'Death Date',
     'element':'DEATH_DT',
    'null': '',
    'universe':dead_mes},     
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        universe = len(ALL[(ALL.type==typo)&(ALL.ME.isin(element['universe']))])
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        true_complete = len(ALL[(ALL.type==typo)&(ALL[element['element']]!=element['null'])&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [87]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=MDM; UID={}; PWD={}".format(username, password_edw)
STG = pyodbc.connect(w)

In [26]:
#abms
BOARD_QUERY = \
    f"""
    SELECT
    PARTY_ID_FROM,
    PARTY_ID_TO,
    ABMS_ID
    FROM
    AMAEDW.BRD_CERTIF
    WHERE
    THRU_DT is null
    """
abms = pd.read_sql(con=AMAEDW, sql=BOARD_QUERY)

abms_person_query = '''
    SELECT 
    AP.PKEY_SRC_OBJECT,
    AP.FRST_NM,
    AP.LST_NM,
    AP.MDL_NM,
    AP.FULL_NM,
    AP.SFX_NM,
    AP.BIRTH_YR,
    AP.BIRTH_DT,
    AP.DEATH_DT,
    AP.RAW_FRST_NM,
    AP.RAW_LST_NM,
    AP.RAW_MDL_NM,
    AP.RAW_FULL_NM,
    AP.PLC_OF_BRTH,
    AP.MORT_STS_CD
    FROM 
    STG.C_S_ABMS_PTY AP
    WHERE 
    AP.DELETED_IND IS NULL
    AND
    AP.HUB_STATE_IND = 1
    '''
abms_person = pd.read_sql(con=STG, sql=abms_person_query)

In [43]:
abms_person['ABMS_ID'] = [float(x.split('~')[1]) for x in abms_person.PKEY_SRC_OBJECT]
abms_all = pd.merge(abms, ALL, left_on='PARTY_ID_FROM', right_on='PARTY_ID')
abms_all = pd.merge(abms_person, abms_all, on='ABMS_ID', suffixes=['_ABMS','_EDW'])
abms_all = abms_all.drop_duplicates()

In [ ]:
abms_dict = abms_all.to_dict('records')

In [63]:
dict_list = []
for row in tqdm(abms_dict):
    birth_dt_match = row['BIRTH_DT_ABMS']==row['BIRTH_DT_EDW']
    death_dt_match = row['DEATH_DT_ABMS']==row['DEATH_DT_EDW']
    last_match = row['RAW_LST_NM']==row['LAST_NM']
    if last_match == False:
        score = fuzz.ratio(row['RAW_LST_NM'],row['LAST_NM'])
    else:
        score = 100
    new_dict = {
        'ME' : row['ME'],
        'BIRTH_DT_MATCH' : birth_dt_match,
        'DEATH_DT_MATCH' : death_dt_match,
        'LAST_MATCH' : last_match,
        'ABMS_LAST': row['RAW_LST_NM'],
        'EDW_LAST': row['LAST_NM'],
        'ABMS_NAME': row['RAW_FULL_NM'],
        'EDW_NAME': row['LABEL_NM'],
        'SCORE': score
        
    }
    dict_list.append(new_dict)

In [75]:
dict_list = []
for row in tqdm(abms_dict):
    first_match = row['FRST_NM']==row['RAW_FRST_NM']
    middle_match = row['MDL_NM']==row['RAW_MDL_NM']
    last_match = row['RAW_LST_NM']==row['LST_NM']
    full_match = row['RAW_FULL_NM']==row['FULL_NM']
    new_dict = {
        'ME' : row['ME'],
        'FIRST_MATCH' : first_match,
        'MIDDLE_MATCH' : middle_match,
        'LAST_MATCH' : last_match,
        'FULL_MATCH' : full_match,
        'RAW_NAME': row['RAW_FULL_NM'],
        'NAME': row['FULL_NM'],
        'EDW_NAME': row['LABEL_NM']
        
    }
    dict_list.append(new_dict)

In [76]:
test = pd.DataFrame(dict_list)

In [79]:
test[~test.MIDDLE_MATCH]

,ME,FIRST_MATCH,MIDDLE_MATCH,LAST_MATCH,FULL_MATCH,RAW_NAME,NAME,EDW_NAME


In [74]:
# test = pd.DataFrame(dict_list)
hmm = list(test[(~test.LAST_MATCH)&(test.SCORE<60)].ME)

In [41]:
abms_all.columns

Index(['PKEY_SRC_OBJECT', 'FRST_NM', 'LST_NM', 'MDL_NM', 'FULL_NM', 'SFX_NM',
       'BIRTH_YR', 'BIRTH_DT_ABMS', 'DEATH_DT_ABMS', 'RAW_FRST_NM',
       'RAW_LST_NM', 'RAW_MDL_NM', 'RAW_FULL_NM', 'PLC_OF_BRTH', 'MORT_STS_CD',
       'BRD_ID', 'ABMS_ID', 'PARTY_ID_FROM', 'PARTY_ID_TO', 'PARTY_ID',
       'ENTITY_ID', 'ME', 'UPDATE_TMSTP', 'ACTIVE_IND',
       'medical_education_number', 'type', 'FIRST_NM', 'MIDDLE_NM', 'LAST_NM',
       'SUFFIX', 'LABEL_NM', 'NM_TYPE', 'BIRTH_DT_EDW', 'BIRTH_CITY_NM',
       'BIRTH_STATE_ID', 'BIRTH_COUNTRY_ID', 'GENDER_CD', 'MORTALITY_STS_CD',
       'DEATH_DT_EDW', 'MED_EDU_NBR'],
      dtype='object')

In [42]:
abms_all[['BIRTH_DT_ABMS','BIRTH_DT_EDW']]

,BIRTH_DT_ABMS,BIRTH_DT_EDW
0,1958-03-10,1958-03-10
1,1958-03-10,1958-03-10
2,1954-07-11,1954-07-11
3,1954-07-11,1954-07-11
4,1954-07-11,1954-07-11
...,...,...
2883953,1979-01-05,1979-01-05
2883954,1991-06-07,1991-06-07
2883955,1972-09-14,1972-09-14
2883956,1972-09-14,1972-09-14


In [70]:
ppd[ppd.ME.isin()]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
116846,01002101625,A,1,1.0,LISA A BURKHARTSMEY MD,BURKHARTSMEY,LISA,ANN,NaN,PULMONARY & CCM PGM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
ppd = use.get_ppd()

INFO:useful_functions:PPD file from Mar 05 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [82]:
# q = "DSN=eprdods; UID={}; PWD={}".format(username, password_edw)
# ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT
        ME,
        FIRST_NAME,
        MIDDLE_NAME,
        LAST_NAME,
        GEN_SUFFIX,
        GENDER,
        PRIMARY_PROF_DESC
        FROM 
        ODS.ODS_IMS_PROFESSIONAL
        WHERE  
        CURRENT_BATCH_FLAG='Y'
        AND
        ME IS NOT NULL
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,GEN_SUFFIX,GENDER,ROLE,PRIMARY_PROF_DESC
0,1431104010,JOHN,PAUL,COTTER,None,M,B,Medical Doctors
1,0120108067,MARY,ASHLEY,CAIN,None,F,B,Medical Doctors
2,0481614019,ROBERT,None,GLADNEY,None,M,B,Medical Doctors
3,0180318053,DANIEL,ALAN,EVANS,None,M,B,Medical Doctors
4,0420101001,MARIELSA,None,RABELO- ACEVEDO,None,F,P,Medical Doctors


In [101]:
license_person_query = '''
    SELECT 
    PKEY_SRC_OBJECT,
    FRST_NM,
    LST_NM,
    MDL_NM,
    FULL_NM,
    SFX_NM,
    BIRTH_YR,
    BIRTH_DT,
    DEATH_DT,
    RAW_FRST_NM,
    RAW_LST_NM,
    RAW_MDL_NM,
    RAW_FULL_NM,
    PLC_OF_BRTH,
    GNDR_CD
    FROM 
    STG.C_S_SLN_PTY
    WHERE 
    DELETED_IND IS NULL
    AND
    HUB_STATE_IND = 1
    '''
license_person = pd.read_sql(con=STG, sql=license_person_query)

In [108]:
license_person[~license_person.SFX_NM.isna()]

,PKEY_SRC_OBJECT,FRST_NM,LST_NM,MDL_NM,FULL_NM,SFX_NM,BIRTH_YR,BIRTH_DT,DEATH_DT,RAW_FRST_NM,RAW_LST_NM,RAW_MDL_NM,RAW_FULL_NM,PLC_OF_BRTH,GNDR_CD
109,MI~4301502897~Medical Doctor,Daniel,Beacher,Robert,None,Beache,NaN,None,None,Daniel,Beacher,Robert,None,None,None
119,MI~4301503089~Medical Doctor,Craig,Butler,D,None,II,NaN,None,None,Craig,Butler,D,None,None,None
182,MI~4301503344~Medical Doctor,Miriam,Harewood-Makola,Atira,None,MD,NaN,None,None,Miriam,Harewood-Makola,Atira,None,None,None
189,MI~4301503234~Medical Doctor,Laura,Honch,Jean,None,M.D.,NaN,None,None,Laura,Honch,Jean,None,None,None
206,MI~4301503382~Medical Doctor,Balreet,Kahlon,K,None,MD,NaN,None,None,Balreet,Kahlon,K,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891232,AK~193806~MED11,Terence,Chun,Ung Hoong,Terence Ung Hoong Chun,,NaN,None,None,Terence,Chun,Ung Hoong,Terence Ung Hoong Chun,None,None
2891233,AK~194106~MED11,Hillel,BenAvi,Ari,Hillel Ari BenAvi,,NaN,None,None,Hillel,BenAvi,Ari,Hillel Ari BenAvi,None,None
2891234,AK~194314~MED11,Christopher,Braga,Cabral,Christopher Cabral Braga,,NaN,None,None,Christopher,Braga,Cabral,Christopher Cabral Braga,None,None
2891325,AK~194064~MED11,Calvin,Wu,,Calvin Wu,,NaN,None,None,Calvin,Wu,,Calvin Wu,None,None


In [110]:
len(ppd)/200000

6.543135

In [ ]:
#license
#dea
#npi
#symphony
#dhc
#datagov